In [1]:
# Loading in all packages from the jupyter environment
import gimmemotifs
from gimmemotifs.preprocessing import combine_peaks
from gimmemotifs.preprocessing import coverage_table
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import seaborn as sns
import subprocess as sp
import qnorm  # either add qnorm to the yml, or run conda install qnorm in the env
#import gseapy as gp
#from gseapy.plot import barplot, dotplot
import itertools
import ipywidgets

os.chdir("/ceph/rimlsfnwi/data/moldevbio/zhou/jsmits/markdown_notebooks/scripts")
from Python_scripts import bedtool_closest
from Python_scripts import coverage_table_normalization
from Python_scripts import genome_TSS_annotation
from Python_scripts import genome_TSS_annotation_prom
from Python_scripts import make_autopct
from Python_scripts import summits_2_regions
from Python_scripts import distance_weight

plt.style.use("classic")
%matplotlib inline
#%load_ext nb_black
#%reload_ext nb_black
os.chdir("/ceph/rimlsfnwi/data/moldevbio/zhou/jsmits/tools/scripts")
from Python_scripts import TSS_to_region

In [2]:
"""
# ipython_exit.py
Allows exit() to work if script is invoked with IPython without
raising NameError Exception. Keeps kernel alive.

Use: import variable 'exit' in target script with 'from ipython_exit import exit'    
"""

import sys
from io import StringIO
from IPython import get_ipython


class IpyExit(SystemExit):
    """Exit Exception for IPython.

    Exception temporarily redirects stderr to buffer.
    """
    def __init__(self):
        # print("exiting")  # optionally print some message to stdout, too
        # ... or do other stuff before exit
        sys.stderr = StringIO()

    def __del__(self):
        sys.stderr.close()
        sys.stderr = sys.__stderr__  # restore from backup


def ipy_exit():
    raise IpyExit


if get_ipython():    # ...run with IPython
    exit = ipy_exit  # rebind to custom exit
else:
    exit = exit      # just make exit importable

In [3]:
class analysis:
    """contains all the first steps of my general analysis, including reading a filepath file, config file, and
    exporting those and the conda environment to a logdir.
    
    required input:
    datapaths_file, a tsv file containing all the other filepaths needed in the subsequent analysis
    config_file, a tsv file containing all type of settings and paramters
    output_dir, where to store the output generated by the analysis, it will generate 
    a log and figure dir in this ouput dir
    """

    def __init__(self, datapaths_file: str, config_file: str, output_dir: str, notebook_file: str):
        self.datapaths_file = datapaths_file
        self.config_file = config_file
        self.output_dir = output_dir
        self.files = pd.read_table(
            self.datapaths_file, sep="\t", comment="#", index_col=0, header = 0
        )
        self.config = pd.read_table(
            self.config_file, sep="\t", comment="#", index_col=0, header = 0
        )
        self.notebook_file = notebook_file
        self.logdir = f"{self.output_dir}/logs"
        self.figdir = f"{self.output_dir}/figures"
        self.trackhubdir = f"{self.output_dir}/trackhub"
        self.tmpdir = f"{self.output_dir}/tmp"
        Path(self.tmpdir).mkdir(parents=True, exist_ok=True)
        Path(self.logdir).mkdir(parents=True, exist_ok=True)
        Path(self.figdir).mkdir(parents=True, exist_ok=True)
        Path(self.trackhubdir).mkdir(parents=True, exist_ok=True)


    def save_settings(self, conda_path):
        """save the config and files to the logs, and make a copy of the conda env that his analysis
        is ran in. Takes a full path to the conda env location as input"""
        #save the conda environment
        !{conda_path} list > {self.logdir}/conda_env.txt
        # save both the files and settings of this run to the output_dir
        self.files.T.to_csv(f"{self.logdir}/filepaths.tsv", header=False, sep="\t")
        self.config.to_csv(f"{self.logdir}/config.tsv", header=False, sep="\t")


In [4]:
class general_bedfile_functions:
    """general functions for working with bedfiles & Chromosome coordinates"""

    def filter_bedfile(
        self,
        list_chroms_to_remove: list,
        output_name: str = None,
        bedfile: str = None,
        bed_object: str = None,
        filter_column: str = "Chrom",
        file_output=False,
        return_output=False,
    ):
        """takes a bedfile with x columns and filter to remove a specific list of strings from a column
        if there is no string given, it will try to use the class object df bed.df instead"""
        if bedfile == None:
            bed_df = self.bed_df
        else:
            bed_df = pd.read_table(bedfile, header=None)

        bed_df = bed_df[
            ~bed_df[filter_column].str.contains("|".join(list_chroms_to_remove))
        ]
        if file_output == True:
            bed_df.to_csv(
                f"{self.output_dir}/{output_name}.bed",
                sep="\t",
                header=False,
                index=False,
            )
        if return_output == True:
            return bed_df

In [5]:
class identify_prom_windows(analysis, general_bedfile_functions):
    """identify TSS based promoter windows
    """

    def __init__(self, *args, **kwargs):
        analysis.__init__(self,*args, **kwargs)

    def filter_gtf_file(
        self,
        gtf_file_name: str = "genome_gtf",
        filter_column: int = 17,
        regex: [str] = '";protein_coding"',
        skip_lines: int = 5,
        rerun: bool = False,
    ):
        """filter a (ensembl) gtf file for only containing data of specific transcripts (e.g. genes)
        name of the gtf file in the filepath file
        column to use for filtering (default 17, for ensembl protein coding)
        regex: regex string to keep.
        skip lines: skip the first x lines for skipping meta data of a gtf file
        rerun: rerun if file already excists
        """
        # first check if the filtered gtf file already exists
        self.filtered_gtf = f"{self.tmpdir}/filtered_annotation.gtf"
        if not os.path.exists(self.filtered_gtf) or rerun == True:
            print("filtering GTF_file")
            gtf_file = self.files.loc[gtf_file_name, :].file_location
            Path(f"{self.filtered_gtf}").touch()
            os.remove(f"{self.filtered_gtf}")
            with open(f"{self.filtered_gtf}", "a") as output_gtf:
                with open(gtf_file) as f:
                    for _ in range(skip_lines):
                        next(f)
                    for full_line in f:
                        line = full_line.strip()
                        line = line.split()
                        if str(line[filter_column]) == regex:
                            output_gtf.write(full_line)
        else:
            print('reusing previously filtered gtf file')
        

    def make_TSS_window(
        self,
        TSS_file: str,
        sense_antisense_column: int = 6,
        gene_name_column: int = 13,
        chrom_columnn: int = 0,
        chromstart_columnn: int = 3,
        chromend_columnn: int = 4,
        region_upstream: int = 2000,
        region_downstream: int = 2000,
        rerun=False,
    ):
        """
        Takes a genome gtf TSS file as input, with column info regarding sense/antisense, chr, chrstart, chrend, and
        finally how far up or downstream the window needs to be elongated.
        and returns a smaller gtf file containing only the coordinates of the specified TSS region and the gene name.
        """
        # first check if the filtered gtf file already exists
        print("windowing TSS file")
        tempfile = f"{self.output_dir}/temp_TSS_window.txt"
        Path(tempfile).touch()
        os.remove(f"{tempfile}")

        with open(f"{tempfile}", "a") as output_gtf:
            with open(TSS_file) as f:
                for line in f:
                    line = line.strip()
                    line = line.split()
                    # calculate the adjusted start and stop coordinates, if smaller than 0, make it 1
                    if line[sense_antisense_column] == "+":
                        chromstart = int(line[chromstart_columnn]) - int(
                            region_upstream
                        )
                        if chromstart <= 0:
                            chromstart = 1
                        chromend = (int(line[chromstart_columnn]) + 1) + int(
                            region_downstream
                        )
                    elif line[sense_antisense_column] == "-":
                        chromstart = int(line[chromend_columnn]) - int(
                            region_downstream
                        )
                        if chromstart <= 0:
                            chromstart = 1
                        chromend = (
                            int(line[chromend_columnn]) - 1 + int(region_upstream)
                        )
                        chromstart_temp = chromstart
                        chromstart = chromend
                        chromend = chromstart_temp
                        

                    output_gtf.write(
                        str(line[chrom_columnn])
                        + "\t"
                        + str(chromstart)
                        + "\t"
                        + str(chromend)
                        + "\t"
                        + str(line[sense_antisense_column])
                        + "\t"
                        + str(line[gene_name_column].rstrip(";"))
                        + "\n"
                    )
        tempfilesorted = f"{self.output_dir}/temp_TSS_window_s.txt"
        Path(tempfilesorted).touch()
        os.remove(f"{tempfilesorted}")

        if region_upstream == 0 & region_downstream == 0:
            self.bed_df = pd.read_table(tempfile, header=None, names = ['Chrom','ChromStart','ChromEnd','strand','gene'])
        else:
            sp.check_call(
                f"nice -5 bedtools sort " f"-i {tempfile} " f"> {tempfilesorted}",
                shell=True,
                
            )
            self.bed_df = pd.read_table(tempfilesorted, header=None, names = ['Chrom','ChromStart','ChromEnd','strand','gene'])
            os.remove(tempfilesorted)
        os.remove(tempfile)

In [6]:
class Identify_differential_CREs(identify_prom_windows):
    """Identify variable CRE elements in my analysis"""

    def __init__(self, *args, **kwargs):
        analysis.__init__(self, *args, **kwargs)

    def quantify_histon_mod(
        self,
        regions,
        bamfiles,
        output_file=None,
        alter_window=False,
        extend_start=0,
        extend_end=0,
        rerun=False,
        colnames="bamfiles",
        coverage_window=200,
    ):
        """quantify the ammount of signal in regions in a bedfile/bedobject within a list of bamfiles.
        Optionally you can modify the chromstart and chromend coordinates of the bedfile using alter_window
        and giving a extend_start and extend end number in bp"""
        print("doing stuff")
        if rerun == True or ((output_file != None) & (not os.path.exists(output_file))):
            print("quantifying histone mod")
            if alter_window == True:
                print("resize regions")
                regions["ChromStart"] = regions["ChromStart"].astype(int) - extend_start
                regions["ChromEnd"] = regions["ChromEnd"].astype(int) + extend_end

            regions.to_csv(
                f"{self.tmpdir}/window_regions_quantwindow.bed",
                sep="\t",
                header=False,
                index=False,
            )

            reads_table = coverage_table(
                peakfile=f"{self.tmpdir}/window_regions_quantwindow.bed",
                datafiles=bamfiles,
                window=int(coverage_window),
                log_transform=False,
                ncpus=int(self.config.loc["ncores", :].vallue),
            )
            

            if colnames != "bamfiles":
                print("replacing column names")
                reads_table.columns = colnames
                
            reads_table = reads_table.drop_duplicates()
            reads_table = reads_table.astype(int)
            reads_table.insert(loc=0, column='region', value=reads_table.index)

            if output_file != None:
                reads_table.to_csv(
                    f"{output_file}",
                    sep="\t",
                    header=True,
                    index=False,
                )
            return reads_table
        
        if rerun == False and ((output_file != None) & (os.path.exists(output_file))):
            print('using previously generated table')
            reads_table = pd.read_table(f"{output_file}",sep="\t",index_col = 0)        
            return reads_table



In [9]:
ID_CREs = Identify_differential_CREs(
    #datapaths_file="/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/merge_peaks/files3.tsv",
    datapaths_file="/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/scANANSE_11032022/ATAC_qquant/files_meta.tsv",
    config_file="/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/scANANSE_11032022/ATAC_qquant/config.tsv",
    output_dir="/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/scANANSE/ATAC_qquant/",
    notebook_file="/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/jupyter_notebooks/Combine_peaks.ipynb",
)

ID_CREs.main_dir = (
    "/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/scANANSE/ATAC_qquant/"
)
# datapaths_file="/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/merge_peaks/files.tsv" for all 11 populations
# datapaths_file="/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/data/lako2021/merge_peaks/files2.tsv" for the "four" cell populations

In [11]:
#Merging peaks and generating individual count files
merged_summits = list()
if not os.path.exists(f"{ID_CREs.output_dir}/accesible_summits_reps.bed"):
    ATAC_BAM = ID_CREs.files[ID_CREs.files.data_type == "ATAC_BAM"]
    ATAC_BAM = ATAC_BAM.file_location
    ATAC_peaks = ID_CREs.files[ID_CREs.files.data_type == "ATAC_peak"]
    ATAC_pfiles = ATAC_peaks.file_location 
    combined_ATAC_peaks = combine_peaks(
        list(ATAC_pfiles),
        #print(list(ATAC_pfiles)),
        ID_CREs.files.loc["genome_path_size", :].file_location,
        ID_CREs.config.loc["ATAC_summit", :].vallue,
        True,
    )
    combined_ATAC_peaks.to_csv(
        f"{ID_CREs.tmpdir}/all_merged_ATAC_peaks.bed",
        sep="\t",
        header=False,
        index=False,
    )

    # quantify the peak intensity per cell type
    cell_types = np.unique(
        ID_CREs.files[ID_CREs.files.data_type == "ATAC_BAM"].cell_type,
        return_index=False,
    )
    print("Merging peaks with the cell types: " + cell_types)
    for cell in cell_types:
        print(cell)
        ATAC_BAM = ID_CREs.files[ID_CREs.files.data_type == "ATAC_BAM"]
        ATAC_BAM = ATAC_BAM[ATAC_BAM.cell_type == cell].file_location
        print("Generating the coverage table for cell population " + ATAC_BAM)
        peak_counts = coverage_table(
             peakfile=f"{ID_CREs.tmpdir}/all_merged_ATAC_peaks.bed",
             datafiles=list(ATAC_BAM),
             window=int(ID_CREs.config.loc["ATAC_summit", :][0]),
             log_transform=False,
             ncpus=int(ID_CREs.config.loc["ncores", :][0]),
         )
        peak_counts = peak_counts[
             ~peak_counts.index.str.contains(
                 "|".join(["GL", "Un", "KI", "MT", "X", "Y"])
             )
         ]
        final_df = peak_counts
        final_df.index.name = "loc"
        final_df.to_csv(
            f"{ID_CREs.tmpdir}/{str(cell)}_covtable.tsv",
            sep="\t",
            header=True,
            index=True,
        )

2022-03-11 14:26:54,086 - INFO - Loading data


['Merging peaks with the cell types: CE'
 'Merging peaks with the cell types: CF'
 'Merging peaks with the cell types: CSSC'
 'Merging peaks with the cell types: Cj'
 'Merging peaks with the cell types: LE'
 'Merging peaks with the cell types: LESC'
 'Merging peaks with the cell types: LSC']
CE
filename
CE_ATAC_BAM    Generating the coverage table for cell populat...
Name: file_location, dtype: object


  0%|          | 0/1 [00:00<?, ?it/s]

Process ForkPoolWorker-13:
Process ForkPoolWorker-20:
Process ForkPoolWorker-8:
Process ForkPoolWorker-7:
Process ForkPoolWorker-22:
Process ForkPoolWorker-12:
Process ForkPoolWorker-19:
Process ForkPoolWorker-14:
Process ForkPoolWorker-16:
Process ForkPoolWorker-18:
Process ForkPoolWorker-3:
Process ForkPoolWorker-9:
Process ForkPoolWorker-15:
Process ForkPoolWorker-17:
Process ForkPoolWorker-2:
Process ForkPoolWorker-6:
Process ForkPoolWorker-1:
Process ForkPoolWorker-23:
Process ForkPoolWorker-24:
Process ForkPoolWorker-21:
Process ForkPoolWorker-11:
Process ForkPoolWorker-10:
Process ForkPoolWorker-4:
Traceback (most recent call last):
Process ForkPoolWorker-5:
  File "/vol/mbconda/julian/envs/jupyter/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/vol/mbconda/julian/envs/jupyter/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/vol/mbconda/julian/envs/jupyter/lib/python3.8/m

In [18]:
# Concatenating all cell files to one final file
import pandas as pd
import glob

path = f"{ID_CREs.tmpdir}/"
all_files = []

cell_types = np.unique(
        ID_CREs.files[ID_CREs.files.data_type == "ATAC_BAM"].cell_type,
        return_index=False,
    )

# Joining all cov table locations into one list
for cell in cell_types:
    all_files = glob.glob(path + f"*covtable.tsv")

print("Joining the individual coverage tables to one")

if not os.path.exists(f"{ID_CREs.tmpdir}/joinedcovtable.tsv"):

    # Joining all files to one dataframe
    dffull = pd.concat((pd.read_table(f) for f in all_files),axis=1)

    # Merging the loc columns to one
    dffull = dffull.groupby(level=0, axis=1).first()

    # Setting loc as the index
    dffull.index = dffull['loc']
    del dffull['loc']

    dffull.to_csv(
                f"{ID_CREs.tmpdir}/joinedcovtable.tsv",
                sep="\t",
                header=True,
                index=True,
            )

    dffull
    # Remove all regions that have a sum of zero counts
    dffull2 = dffull.loc[~(dffull==0).all(axis=1)]
    dffull2.to_csv(
                f"{ID_CREs.tmpdir}/joinedcovtable_no_zero.tsv",
                sep="\t",
                header=True,
                index=True,
            )
    dffull2

Joining the individual coverage tables to one
